In [ ]:
import os
import openai
import pinecone
import langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

from dotenv import load_dotenv
load_dotenv()

In [ ]:
def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

In [ ]:
dirToLoad = 'data/ios-example-lt64kB'
documents = load_docs(dirToLoad)
len(documents)

In [ ]:
# Splitting text
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  texts = text_splitter.split_documents(documents)
  return texts

texts = split_docs(documents)
print(len(texts))

In [ ]:
# Getting embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [ ]:
pinecone.init(
    api_key=os.getenv('PINECODE_API_KEY'),
    environment=os.getenv('PINECODE_ENVIRONMENT')
)

In [ ]:
print(pinecone.list_indexes())

In [ ]:
index_name = "ai-repo-reader-1"

In [ ]:
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

In [ ]:
pinecone.create_index(index_name, dimension=1536)

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "What did Alice found in the little glass box?"
docs = docsearch.similarity_search(query, include_metadata=True)
chain.run(input_documents=docs, question=query)